In [18]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense


In [19]:
data_path = '/content/shakespeare_plays2.csv'
df = pd.read_csv(data_path)

In [26]:
df.head()

,Unnamed: 0,play_name,genre,character,act,scene,sentence,text,sex
0,0,All's Well That Ends Well,Comedy,Countess,1,1,1,"In delivering my son from me, I bury a second ...",female
1,1,All's Well That Ends Well,Comedy,Bertram,1,1,2,"And I in going, madam, weep o'er my father's d...",male
2,2,All's Well That Ends Well,Comedy,Bertram,1,1,3,"anew: but I must attend his majesty's command, to",male
3,3,All's Well That Ends Well,Comedy,Bertram,1,1,4,"whom I am now in ward, evermore in subjection.",male
4,4,All's Well That Ends Well,Comedy,Lafeu,1,1,5,"You shall find of the king a husband, madam; you,",male


In [27]:
df.shape

(999, 9)

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 999 entries, 0 to 998
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  999 non-null    int64 
 1   play_name   999 non-null    object
 2   genre       999 non-null    object
 3   character   999 non-null    object
 4   act         999 non-null    int64 
 5   scene       999 non-null    int64 
 6   sentence    999 non-null    int64 
 7   text        999 non-null    object
 8   sex         999 non-null    object
dtypes: int64(4), object(5)
memory usage: 70.4+ KB


In [20]:
texts = df['text'].dropna().tolist()

In [21]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
vocab_size = len(tokenizer.word_index) + 1

In [22]:
input_sequences = []
for line in texts:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        input_sequences.append(token_list[:i+1])

In [23]:
max_sequence_length = max(len(seq) for seq in input_sequences)
input_sequences = pad_sequences(input_sequences, maxlen=max_sequence_length, padding='pre')

In [24]:
X, y = input_sequences[:, :-1], input_sequences[:, -1]
y = tf.keras.utils.to_categorical(y, num_classes=vocab_size)

In [25]:
model = Sequential([
    Embedding(vocab_size, 128, input_length=max_sequence_length-1),
    LSTM(128, return_sequences=True),
    LSTM(128),
    Dense(128, activation='relu'),
    Dense(vocab_size, activation='softmax')
])


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [30]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [31]:
history = model.fit(X, y, epochs=10, batch_size=64, verbose=1)

Epoch 1/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 13s 82ms/step - accuracy: 0.0162 - loss: 7.0148
Epoch 2/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 7s 71ms/step - accuracy: 0.0216 - loss: 6.3115
Epoch 3/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 9s 82ms/step - accuracy: 0.0231 - loss: 6.1388
Epoch 4/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 10s 83ms/step - accuracy: 0.0295 - loss: 6.0533
Epoch 5/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 10s 85ms/step - accuracy: 0.0352 - loss: 5.9063
Epoch 6/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 9s 70ms/step - accuracy: 0.0386 - loss: 5.8074
Epoch 7/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 9s 82ms/step - accuracy: 0.0398 - loss: 5.7533
Epoch 8/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 10s 83ms/step - accuracy: 0.0412 - loss: 5.6954
Epoch 9/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 8s 77ms/step - accuracy: 0.0475 - loss: 5.6594
Epoch 10/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 9s 69ms/step - accuracy: 0.0440 - loss: 5.5776


In [39]:
import numpy as np

def generate_text(seed_text, next_words=50, temperature=1.0):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_length-1, padding='pre')

        predictions = model.predict(token_list)[0]

        predictions = tf.nn.softmax(predictions / temperature).numpy()
        predictions = np.clip(predictions, 1e-8, 1.0 - 1e-8)
        predictions = predictions / np.sum(predictions)

        predicted_index = np.random.choice(range(vocab_size), p=predictions)

        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted_index:
                output_word = word
                break

        seed_text += " " + output_word
    return seed_text


In [43]:
print(generate_text("The", next_words=20, temperature=0.8))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 170ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 366ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 240ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
The fruitfully doctrine skill foul o' disaster down parolles barr'd encounter breed floods further spare raised curls act off's bidding play
